# GPT1-3

[李沐讲论文](https://www.youtube.com/watch?v=t70Bl3w7bxY&list=PLFXJ6jwg0qW-7UM8iUTj3qKqdhbQULP5I&index=18)

## GPT1: 
![](01.png)

![](02.png)

1. 之所以用decoder，是指的利用其中的mask，把当前以及后面的词屏蔽掉

![](03.png)

1. L1 就是前面pre-trainning的L。也就是在tuning的时候，同时也训练前面训练过得transformer模型的参数。同时把这两个目标函数做个结合

![](04.png)

1. `Extract`,`Delim`,`Start`都是特殊的token，用于最后的分类等任务

## GPT2
![](05.png)

![](06.png)


1. McCann 就是想用一个模型来完成多个任务。所以在训练模型的时候，就把要做的任务也作为input输入到模型中，构造新的序列去训练。像文中的`translate to french`,`answer the question`。这种也就是所谓的prompt

![](07.png)

1. GPT2则更进了一步，认为文本中已经包含了这些prompt的信息。只要模型足够大，就能够学习到这种关系。比如那个例子。文本中有些英语翻译法语的例子，模型通过读取这些信息，就能够学会英语翻译法语。而这些句子中间的其他词，也就是prompt，无需人为构造

## GPT3
![](08.png)


![](09.png)
![](10.png)

1. 传统的fine-tuning，就是用有标记的数据，去训练pre-train的模型，同时跟新其参数

2. GPT则采用了上下文学习，不去更新其权重（模型太大了）。这种基于上下文的学习(in-context learning)，就是采用提供一个指令，以及一些example，直接让模型输出结果。而这种能力，正是基于transformer和大量文本学习得到的

![](11.png)

1. 在处理Common Crawl脏数据的，时候，其中，1是说的用reddit数据作为正数据集，common crawl的作为负数据集。然后训练一个Logistic Regression二分类器。然后用这个分类器去预测所有common crawl的结果，如果结果偏正，就纳入训练集。NICE！

## InstructGPT

[李沐讲论文](https://www.youtube.com/watch?v=zfIGAwD1jOQ&list=RDCMUC8WCW6C3BWLKSZ5cMzD8Gyw&index=8)

![](12.png)

1. InstructGPT 就是在原始GTP的基础上，用一些人工标注的prompts去做了fine tunning

![](13.png)

整个模型分为三部分

1. 让人手写了一些prompts和对应的输出，用GTP3为基础，作了监督微调

![](14.png)



2. 用前面训练的SFT为基础，只不过把最后的输出层转换成了一个输出一个数的打分层

![](15.png)

训练这个模型的目的，是为了后面强化学习提供一个环境，给强化学习目标反馈。之所以要单独训练这样一个模型，是为了将人类反馈和强化学习分离开，做到离线学习的目的。这里用到的数据集是他们收集的第二个数据集，人类排序的数据集。

损失函数用的典型的pairwise的loss。对于每一对输出$(y_w, y_l)$，在计算loss的时候，如果人类的排序，w比l好，就把$y_w$ 放到前面，否则就把$y_l$放到前面。这样优化目标就是最大化loss function。就能让模型学到这种ranking关系

3. 强化学习模型，用前面的RM模型作为环境反馈，继续以SFT模型作为基础作强化训练

![](16.png)

这里的目标函数除了前面RM模型（环境）的反馈值，还加了一项KL散度的惩罚项（$-\beta log (..) $）。目的是让最终学习到的概率分布不会比原来的差别过大，防止过拟合。

最后一项是原来预训练GPT3的目标函数也加上了。所以公式前半部分，都是在新数据集上做得拟合，而后半部分则是原先的拟合目标。保证新训练的模型在原来问题上不会衰减太多
